# Functional Test 3.1.1 - creating simple VMs of varying sizes

This Jupyter notebook will allow you to create VMs of different sizes on different sites and worker nodes consistent with requirements for test 3.1.1.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## Step 3 Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [ ]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the test Slice

The following creates a single node with basic compute capabilities at a specific site on a specific worker. Modify the site, worker and the dimensions of the desired VM. 

**RAM and disk are measured in GB.**

**Note that FABRIC should give you a larger VM than you are asking up to a maximum limit.** 

**Pay attention to how many cores, RAM and Disk are actually allocated.**

**The code to create the slice will auto-refresh until the slice is created or it fails**

In [ ]:
from datetime import datetime
from dateutil import tz

site='STAR'
# how many workers site has
worker_cnt=6

# if you want to isolate the test to a specific subset of workers populate this list
# with their indices
only_workers = []

cores=10
ram=20
disk=50
slice_name=f"Slice Test 3.1.1 {site} on {worker} on {datetime.now()}"

# create a dictionary node->worker name so we start one in each worker
worker_range = only_workers if len(only_workers) > 0 else range(1, worker_cnt + 1)
nodes = { f'Node{i}': f'{site.lower()}-w{i}.fabric-testbed.net' for i in worker_range}
print(f'{nodes=}')

In [ ]:
try:
    # Create a slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Add a node
    for n,w in nodes.items():
        print(f'Adding node {n} to worker {w}')
        slice.add_node(name=n, site=site, host=w, cores=cores, ram=ram, disk=disk)
    
    # Submit the slice
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Step 5: Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

### Print the node

Each node in the slice has a set of get functions that return the node's attributes. Use the returned `SSH Command` string to check the node. You can do it from a Bash launched inside the Jupyter container.

In [ ]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")